# 3DP Dissertation Experiments

## Fractional Factorial Experiments

**Process Parameters:**

- Layer Thickness (LH)
- Print Speed (PS)
- Infill Density (ID)
- Raster Width (RW) - Infill Line Width in Cura
- Wall Thickness (WT)




In [4]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as stats


from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from scipy import stats

%matplotlib inline
PlotWidth = 6

import warnings
warnings.filterwarnings('ignore')

## Print Parameter Variables:

- **lh_low** = Layer Thickness Lower Limit (mm)
- **lh_hi** = Layer Thickness Upper Limit (mm)
- **ps_low** = Print Speed Lower Limit (mm/s)
- **ps_hi** = Print Speed Upper Limit (mm/s)
- **id_low** = Infill Density Lower Limit (%)
- **id_hi** = Infill Density Upper Limit (%)
- **rw_low** = Raster Width Lower Limit (mm)
- **rw_hi** = Raster Width Upper Limit (mm)
- **wt_low** = Wall Thickness Lower Limit (mm)
- **wt_hi** = Wall Thickness Upper Limit (mm)

In [5]:
#@title Default title text
lh_low = .16 #@param {type:"number"}
lh_hi = .24 #@param {type:"number"}
ps_low = 50 #@param {type:"number"}
ps_hi = 60 #@param {type:"number"}
id_low = .25 #@param {type:"number"}
id_hi = .15 #@param {type:"number"}
rw_low = .4 #@param {type:"number"}
rw_hi = .8 #@param {type:"number"}
wt_low = 1.2 #@param {type:"number"}
wt_hi = .8 #@param {type:"number"}

In [6]:
# set some variables related to the coffee data set
actual_lows = { 'layer_thickness' : lh_low, 'print_speed' : ps_low, 'infill_density': id_low, 'raster_width': rw_low, 'wall_thickness': wt_low }
actual_highs = { 'layer_thickness' : lh_hi, 'print_speed' : ps_hi, 'infill_density': id_hi, 'raster_width': rw_hi, 'wall_thickness': wt_hi }
units = { 'layer_thickness' : 'mm', 'print_speed' : 'mm/s', 'infill_density': '%', 'raster_width': 'mm', 'wall_thickness': 'mm' }

In [7]:
columns = ['Trial Number', 'Layer Thickness', 'Print Speed', 'Infill Density', 'Raster Width', 'Wall Thickness','Cost','Time', 'Quality']
myDoE = pd.read_csv('doe-schedule.csv', skiprows=1, names=columns)

display(myDoE)

FileNotFoundError: [Errno 2] No such file or directory: 'doe-schedule.csv'

In [ ]:
myDoE['Layer Thickness'] = myDoE['Layer Thickness'].replace([-1,1], [lh_low,lh_hi])
myDoE['Print Speed'] = myDoE['Print Speed'].replace([-1,1], [ps_low,ps_hi])
myDoE['Infill Density'] = myDoE['Infill Density'].replace([-1,1], [id_low,id_hi])
myDoE['Raster Width'] = myDoE['Raster Width'].replace([-1,1], [rw_low,rw_hi])
myDoE['Wall Thickness'] = myDoE['Wall Thickness'].replace([-1,1], [wt_low,wt_hi])
display(myDoE)

# CR6 SE Results

In [ ]:
cr6_results_pd = pd.read_csv('cr6_results.csv')
cr6_results_pd.head()

In [ ]:
np.random.seed(1234)
dataNew=pd.DataFrame({'CCR6SE-Default.gcode':random.sample(list(cr6_results_pd['Used Filament Cost'][cr6_results_pd['File Name']=='CCR6SE-Default.gcode']), 1),
                      'CCR6SE_Run2.gcode':random.sample(list(cr6_results_pd['Used Filament Cost'][cr6_results_pd['File Name']=='CCR6SE_Run2.gcode']), 1),
                      'CCR6SE_Run3.gcode':random.sample(list(cr6_results_pd['Used Filament Cost'][cr6_results_pd['File Name']=='CCR6SE_Run3.gcode']), 1),
                      'CCR6SE_Run4.gcode':random.sample(list(cr6_results_pd['Used Filament Cost'][cr6_results_pd['File Name']=='CCR6SE_Run4.gcode']), 1)})
display(dataNew)

# Cost Analysis

In [ ]:
y_cost = myDoE['Cost']
X = myDoE[['Layer Thickness', 'Print Speed', 'Infill Density', 'Raster Width', 'Wall Thickness']]

## An intercept is not added by default, so we need to add that here
X = sm.add_constant(X)
cost_results = sm.OLS(y_cost, X).fit()
cost_results.summary()

print(cost_results.summary())

:
## p-value is more than 0.05 , So we can say that variances among groups are equal.

In [ ]:

F, p = stats.f_oneway(myDoE['Layer Thickness'],myDoE['Print Speed'],myDoE['Infill Density'],myDoE['Raster Width'],myDoE['Wall Thickness'])
print('F statistic =', F, 'p value :',p)

# Seeing if the overall model is significant

In [ ]:
F, p = stats.f_oneway(myDoE['Layer Thickness'],myDoE['Print Speed'],myDoE['Infill Density'],myDoE['Raster Width'],myDoE['Wall Thickness'])
# Seeing if the overall model is significant
print('F-Statistic=%.3f, p=%.3f' % (F, p))

In [ ]:
df_melt = pd.melt(myDoE.reset_index(), id_vars=['index'], value_vars=['Layer Thickness', 'Print Speed', 'Infill Density', 'Raster Width', 'Wall Thickness'])
display(df_melt)

# Time Analysis

In [ ]:
y_time = myDoE['Time']
time_results = sm.OLS(y_time, X).fit()
time_results.summary()

print(time_results.summary())

# Quality Analysis

In [ ]:
y_qual = myDoE['Quality']
qual_results = sm.OLS(y_qual, X).fit()
qual_results.summary()

print(qual_results.summary())